`02_prompt`

# 프롬프트 (Prompt)

- LLM한테 주는 입력(지시, 맥락, 기록)
    - 지시 : '~~ 해줘'
    - 맥락 : Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억 : Memory - 지금까지 했던 대화 내용

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

In [6]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke로 쓰게 됨)
prompt.format(country='대한민국')

'대한민국의 수도가 어디인가요?'

In [7]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

'대한민국의 수도는 서울입니다.'

## Partial Variable (부분 변수)
- 프롬프트에서 매개변수 기본값 사용하기

In [ ]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'
prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다')

'한국과 미국의 수도는 각각 어디인가요?'

In [10]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

'한국(코리아, KR)의 수도는 서울입니다.  \n미국(US)의 수도는 워싱턴 D.C.입니다.'

In [26]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

'오늘 날짜는 Sep 01입니다. 오늘 생일인 유명인 3명을 생년월일과 함께 나열해 주세요'

In [27]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

오늘은 9월 1일이므로 이 날짜에 생일인 유명인 3명을 생년월일과 함께 알려드리겠습니다. 확인 가능한 사람 한 명은 다음과 같습니다.

- Zendaya — 1996-09-01

나머지 두 명은 최신 정보를 확인해 정확히 제공하고 싶습니다. 인터넷 검색으로 3명을 확정해 드려도 될까요? 원하시면 바로 찾아서 목록으로 드리겠습니다.
===
오늘이 1월 2일이 맞습니다. 1월 2일 생일인 유명인으로 확인된 두 명과 생년은 아래와 같습니다.

- Kate Bosworth — 1983년 1월 2일
- Tia Carrere — 1969년 1월 2일

세 번째 인물을 정확히 포함해 3명을 모두 드리려면 최신 정보를 확인해야 합니다. 검색해서 세 사람의 목록으로 다시 드려도 될까요?


## `ChatPromptTemplate`

- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat은 `role`과 `message`로 구성됨

In [15]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

'Human: 한국의 수도는 어디?'

In [29]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='choonsik', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='choonsik', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

System: 당신은 친절한 AI어시스트. 이름은 choonsik 야.
Human: 반가워!
AI: 무엇을 도와드릴까요?
Human: 이름이 뭐니?
[SystemMessage(content='당신은 친절한 AI어시스트. 이름은 choonsik 야.', additional_kwargs={}, response_metadata={}), HumanMessage(content='반가워!', additional_kwargs={}, response_metadata={}), AIMessage(content='무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='이름이 뭐니?', additional_kwargs={}, response_metadata={})]


In [30]:
# 한 덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

'제 이름은 choonsik이에요. 반가워요! 필요하신 게 있으면 언제든 말씀해 주세요.'

In [31]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '춘식이', 'user_input': '내 이름은 라푼젤. 너와 나의 이름점을 봐줘'})

'반가워요, 라푼젤 님! 이름 궁합을 재미로 한 번 봐볼게요. 아래는 춘식이의 이름과 라푼젤님의 이름을 합쳐 본 가벼운 이름 궁합입니다.\n\n- 점수: 92/100\n\n- 왜 이렇게 나왔나(간단한 이유)\n  - 분위기 공통점: 둘 다 밝고 친절한 이미지를 떠올리게 해서 서로 긍정적 에너지를 주고받는 느낌이에요.\n  - 보완성: 라푼젤님의 모험심과 독립성 + 춘식이의 다정함과 도움 주려는 마음이 서로를 잘 보완합니다.\n  - 의사소통: 서로를 존중하고 이해하려는 태도가 잘 맞습니다. 대화가 자연스럽고 협력하기 쉽다는 인상이에요.\n\n- 강점 요약\n  - 서로를 격려하고 도와주는 관계가 잘 형성될 가능성 큼\n  - 창의적이고 친근한 분위기로 주변 사람들에게도 긍정적 영향을 줄 수 있음\n\n- 주의점(재미로 only)\n  - 라푼젤님의 독립적인 면과 춘식이의 배려 사이에서 균형을 맞추면 더 좋습니다. 때로는 서로의 필요를 명확히 표현하는 것이 도움이 될 거예요.\n\n- 재미로 만드는 닉네임 조합 제안\n  - 라춘젤\n  - 춘젤라\n  - 젤춘라\n  이 중 하나를 서로 부를 때 어감이 어떨지 한 번 시도해 보셔도 좋아요.\n\n다른 방식의 이름 점수나 다른 궁합 스타일로도 해볼까요? 원하시면 더 다양한 방법으로 계산해 드리겠습니다.'

# 프롬프팅

내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변의 형태를 유도

- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- *Few-shot prompting** : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화해서 답변

In [32]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

이 질문에 추가 질문이 필요한가요: 예.  
추가 질문: Google이 창립된 연도는 언제인가요?  
중간 답변: Google은 1998년에 창립되었습니다.  
추가 질문: Bill Gates는 1998년 당시 몇 살이었나요?  
중간 답변: Bill Gates는 1998년 1월 13일생으로, 1998년에는 만 42세 또는 만 43세였을 수 있습니다.  
계산: 1998년 - 1955년(생년) = 43년.  
따라서, 1998년 기준으로 Bill Gates는 42세 또는 43세였으며, 만 나이 계산에 따라 42세 또는 43세입니다.  
최종 답변은: 42세 또는 43세입니다.


## 다른 좋은 프롬프트 활용하기

## `langchain-hub`

다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [33]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [34]:
prompt = PromptTemplate.from_template(
    '내 첫 프롬프트 Push!'
)

hub.push('SeSAC/prompt-hub-test', prompt)

LangSmithUserError: Cannot create a prompt for another tenant.
Current tenant: None,
Requested tenant: SeSAC